Ta sẽ cho dữ liệu của rotten tomato vào data warehouse, với schema được thể hiện như sau:

Bảng Movie Info: chứa các thông tin chung của movie

- movie_id: mã phim, được mã hoá dưới dạng số
- movie_title: tiêu đề
- movie_info: mô tả
- content_rating: giới hạn độ tuổi xem
- genres: thể loại
- directors: đạo diễn bộ phim
- actors: diễn viên tham gia bộ phim
- release_date: ngày ra mắt
- runtime: thời lượng bộ phim

Bảng Tomato Rating Info: chứa các thông tin về rating của movie trên trang Rotten Tomato
- movie_id: khoá ngoại
- tomatometer_rating: rating trung bình của các nhà phê bình phim
- tomatometer_count: số lượng rating của các nhà phê bình phim
- audience_rating: rating trung bình của khán giả
- audience_count: số lượng rating của khán giả
- link: link của trang web dẫn đến phim

Với các nguồn dữ liệu khác khi cho vào data warehouse cũng sẽ tách ra làm 2 bảng riêng: bảng về thông tin chung về movie và bảng về thông tin rating của movie đối với nguồn đó

Ngoài ra, còn một số bảng tham khảo các giá trị của các trường trong data warehouse thể hiện ở dưới (Phục vụ cho xử lý các nguồn dữ liệu khác):

- Bảng Cast and Director: chứa danh sách các đạo diễn và diễn viên tham gia phim
- Bảng Production Company: chứa danh sách các nhà sản xuất
- Bảng Genres: chứa danh sách các thể loại
- Bảng Content Rating: chứa danh sách các content rating

In [42]:
import pandas as pd

In [49]:
# Movie Info
df_tomato_movie = pd.read_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/rotten_tomatoes/rotten_tomatoes_movies.csv")
df_tomato_movie.head()

,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,original_release_date,...,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,...,20th Century Fox,Rotten,49.0,149.0,Spilled,53.0,254421.0,43,73,76
1,m/0878835,Please Give,Kate (Catherine Keener) and her husband Alex (...,Nicole Holofcener's newest might seem slight i...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",2010-04-30,...,Sony Pictures Classics,Certified-Fresh,87.0,142.0,Upright,64.0,11574.0,44,123,19
2,m/10,10,"A successful, middle-aged Hollywood songwriter...",Blake Edwards' bawdy comedy may not score a pe...,R,"Comedy, Romance",Blake Edwards,Blake Edwards,"Dudley Moore, Bo Derek, Julie Andrews, Robert ...",1979-10-05,...,Waner Bros.,Fresh,67.0,24.0,Spilled,53.0,14684.0,2,16,8
3,m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),Following the closing arguments in a murder tr...,Sidney Lumet's feature debut is a superbly wri...,NR,"Classics, Drama",Sidney Lumet,Reginald Rose,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",1957-04-13,...,Criterion Collection,Certified-Fresh,100.0,54.0,Upright,97.0,105386.0,6,54,0
4,m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea","In 1866, Professor Pierre M. Aronnax (Paul Luk...","One of Disney's finest live-action adventures,...",G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,Earl Felton,"James Mason, Kirk Douglas, Paul Lukas, Peter L...",1954-01-01,...,Disney,Fresh,89.0,27.0,Upright,74.0,68918.0,5,24,3


# Tạo bảng trong data warehouse

In [50]:
# Loại bỏ các trường xuất hiện trong warehouse schema
df_tomato_movie.drop(['critics_consensus', 'authors', 'streaming_release_date', 
                      'tomatometer_status', 'audience_status', 'tomatometer_top_critics_count', 
                      'tomatometer_fresh_critics_count', 'tomatometer_rotten_critics_count', 'production_company'], axis=1, inplace=True)

In [51]:
# Tạo một trường movie_id
df_tomato_movie["movie_id"]=["movie"+str(idx) for idx in range(len(df_tomato_movie))]
# Chỉnh sửa trường rotten_tomatoes_link
df_tomato_movie["rotten_tomatoes_link"]=["https://www.rottentomatoes.com/"+i for i in df_tomato_movie["rotten_tomatoes_link"]]

In [52]:
# Thay đổi tên một số trường
df_tomato_movie.rename(columns = {'rotten_tomatoes_link':'link', 'original_release_date':'release_date'}, inplace = True)
df_tomato_movie.columns

Index(['link', 'movie_title', 'movie_info', 'content_rating', 'genres',
       'directors', 'actors', 'release_date', 'runtime', 'tomatometer_rating',
       'tomatometer_count', 'audience_rating', 'audience_count', 'movie_id'],
      dtype='object')

In [53]:
df_warehouse_movie = df_tomato_movie[["movie_id", "movie_title", "movie_info", "genres", 
                                      "directors", "actors", "release_date", "runtime", "content_rating"]]
df_warehouse_tomato_rating = df_tomato_movie[["movie_id", "tomatometer_rating", "tomatometer_count", 
                                              "audience_rating", "audience_count", "link"]]

# Tạo các bảng tham khảo

## Trường content_rating

In [8]:
df_warehouse_movie['content_rating'].unique()

array(['PG', 'R', 'NR', 'G', 'PG-13', 'NC17'], dtype=object)

content_rating description:

- G: GENERAL AUDIENCES: ALL AGES ADMITTED
- PG: PARENTAL GUIDANCE SUGGESTED: SOME MATERIAL MAY NOT BE SUITABLE FOR CHILDREN
- PG-13: PARENTS STRONGLY CAUTIONED: SOME MATERIAL MAY BE INAPPROPRIATE FOR CHILDREN UNDER 13
- R: RESTRICTED: UNDER 17 REQUIRES ACCOMPANYING PARENT OR ADULT GUARDIAN
- NC-17: NO ONE 17 AND UNDER ADMITTED
- NR: NOT RATED: THE CONTENT OF THIS FILM HAS NOT BEEN EVALUATED (TRAILER)

In [9]:
pd.DataFrame(list(df_warehouse_movie['content_rating'].unique()), columns=["content_rating"]).to_csv(
    "/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/warehouse/field_value/content_rating.csv",
    index=False 
)

## Trường genres

Cấu trúc giá trị trong trường genres là <genre 1>,  <genres 2>, ...

In [11]:
genres_list = []
for i in df_warehouse_movie["genres"]:
    if type(i)==str:
        genres_items = i.split(", ")
        genres_list += genres_items
genres_list = list(set(genres_list))

In [14]:
genres_list

['Art House & International',
 'Sports & Fitness',
 'Television',
 'Science Fiction & Fantasy',
 'Comedy',
 'Musical & Performing Arts',
 'Anime & Manga',
 'Mystery & Suspense',
 'Drama',
 'Cult Movies',
 'Western',
 'Faith & Spirituality',
 'Horror',
 'Kids & Family',
 'Documentary',
 'Animation',
 'Classics',
 'Special Interest',
 'Action & Adventure',
 'Gay & Lesbian',
 'Romance']

In [34]:
pd.DataFrame(genres_list, columns=["genres"]).to_csv(
    "/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/warehouse/field_value/genres.csv",
    index=False 
)

## Trường directors và actors

Cấu trúc giá trị của trường director <director 1>, <director 2>, ...

Cấu trúc giá trị của trường actor <actor 1>, <actor 2>, ...


In [63]:
name_list = []
for i in df_warehouse_movie["directors"]:
    if type(i)==str:
        name_items = i.split(", ")
        name_list += name_items
for i in df_warehouse_movie["actors"]:
    if type(i)==str:
        name_items = i.split(", ")
        name_list += name_items
name_list = list(set(name_list))
name_list.sort()

In [39]:
len(name_list)

210926

In [40]:
pd.DataFrame(name_list, columns=["name"]).to_csv(
    "/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/warehouse/field_value/directors_actors.csv",
    index=False 
)

# Lưu dữ liệu data warehouse

In [54]:
df_warehouse_movie.to_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/warehouse/movie.csv", index=False)
df_warehouse_tomato_rating.to_csv("/Users/trananhvu/Documents/Tichhopdulieu/Data_Integration_Group23/Data/warehouse/tomato_rating.csv", index=False)